# Notebook 02 - Standardize Columns and Export Clean Tables

This notebook loads the raw snapshot files, corrects header offsets, and applies consistent column naming across all datasets. The goal is to ensure uniform structure across files for merging, translation, and matching tasks in later notebooks.

We rename ambiguous fields (e.g. `Unnamed: 3`) and export each cleaned dataset to CSV format under `cleaned_data/`.


In [1]:
import pandas as pd
import os

# Raw and output paths
data_folder = r"C:\Users\User\Desktop\University\BEP\Data\Dirk data"
output_folder = "cleaned_data"
os.makedirs(output_folder, exist_ok=True)


## Cleaning Strategy

We apply the following principles across datasets:

- Header offset is handled using `skiprows` (as determined in Notebook 01)
- Column names are:
  - Lowercased
  - Stripped of whitespace
  - Converted to snake_case
- Known ambiguous or unnamed columns are renamed manually
- A `product_name_clean` column is added to the waste dataset for semantic matching

We now begin cleaning each file individually.


In [2]:
# Load waste data (correct header row)
waste_path = os.path.join(data_folder, "2025-01-24T07_01_23+00_00zero waste lab Mark Down_Waste 2025-01-24.xlsx")
df_waste = pd.read_excel(waste_path, sheet_name=0, skiprows=2)

# Preview raw columns
print("Original columns:")
print(df_waste.columns.tolist())


Original columns:
['Store', 'Date', 'Article', 'Unnamed: 3', 'Product name', 'Brand', 'Content', 'Eenheid CE', 'Supplier', 'Unnamed: 9', 'Content category', 'Waste reason', 'Items wasted', 'Value wasted']


In [3]:
# Create a copy with renamed and standardized columns
df_waste_cleaned = df_waste.rename(columns={
    "Store": "store",
    "Date": "date",
    "Article": "article",
    "Product name": "product_name",
    "Brand": "brand",
    "Content": "content",
    "Eenheid CE": "unit",
    "Supplier": "supplier",
    "Content category": "content_category",
    "Waste reason": "waste_reason",
    "Items wasted": "items_wasted",
    "Value wasted": "value_wasted"
})

# Drop irrelevant unnamed columns
df_waste_cleaned = df_waste_cleaned.drop(columns=[col for col in df_waste_cleaned.columns if "Unnamed" in col])

# Clean product name for canonical usage
def clean_product_name(name):
    if isinstance(name, str):
        return name.lower().strip()
    return name

df_waste_cleaned["product_name_clean"] = df_waste_cleaned["product_name"].apply(clean_product_name)


In [4]:
# Export cleaned version
waste_out_path = os.path.join(output_folder, "waste_snapshot_cleaned.csv")
df_waste_cleaned.to_csv(waste_out_path, index=False)

print("Waste data cleaned and saved to:", waste_out_path)


Waste data cleaned and saved to: cleaned_data\waste_snapshot_cleaned.csv


## Clean Mark Down Discount File

This dataset contains products that were discounted at each store. It includes article identifiers, discount percentage, and regular price per product. We correct for header offset, remove unnamed columns, and standardize naming conventions to align it with the waste dataset.

Key columns we aim to standardize:
- `Filiaal` -> `store`
- `Date`, `Time`, `Article` -> `date`, `time`, `article`
- `Discount percentage` -> `discount_percentage`
- `Regular price` -> `regular_price`


In [5]:
# Load markdown file
markdown_path = os.path.join(data_folder, "2025-01-24T07_03_00+00_00zero waste lab Mark Down 2025-01-24.xlsx")
df_markdown = pd.read_excel(markdown_path, sheet_name=0, skiprows=2)

# Rename columns
df_markdown_cleaned = df_markdown.rename(columns={
    "Filiaal": "store",
    "Date": "date",
    "Time": "time",
    "Article": "article",
    "Discount percentage": "discount_percentage",
    "Regular price": "regular_price",
    "Pakking price": "packing_price",
    "total amount discounted": "discounted_quantity"
})

# Drop generic unnamed columns
df_markdown_cleaned = df_markdown_cleaned.drop(columns=[col for col in df_markdown_cleaned.columns if "Unnamed" in col])


In [7]:
markdown_out_path = os.path.join(output_folder, "markdown_snapshot_cleaned.csv")
df_markdown_cleaned.to_csv(markdown_out_path, index=False)

print(" Markdown data cleaned and saved to:", markdown_out_path)


 Markdown data cleaned and saved to: cleaned_data\markdown_snapshot_cleaned.csv


## Clean Sales Data File (Part I)

This dataset contains transaction-level sales records for a single day. It includes product identifiers, pricing information, promotion status, and quantities sold.

We apply the same conventions used for the waste and markdown datasets. Key columns to standardize:

- `Store`, `Date`, `Article` -> `store`, `date`, `article`
- `Discount 0/1` -> `discount_flag`
- `Promotion` -> `promotion`
- `Theoretische Kassaverkoopprijs` -> `price_theoretical`
- `Selling price` -> `price_sold`
- `Items`, `Volume`, `Sold value` -> `items_sold`, `volume_sold`, `revenue_sold`


In [8]:
# Load sales data I
sales_path = os.path.join(data_folder, "2025-01-24T07_02_33+00_00zero waste lab_Salesdata I 2025-01-24.xlsx")
df_sales = pd.read_excel(sales_path, sheet_name=0, skiprows=2)

# Rename columns
df_sales_cleaned = df_sales.rename(columns={
    "Store": "store",
    "Date": "date",
    "Article": "article",
    "product category": "product_category",
    "Discount 0/1": "discount_flag",
    "Promotion": "promotion",
    "Theoretische Kassaverkoopprijs": "price_theoretical",
    "Selling price": "price_sold",
    "items": "items_sold",
    "Volume": "volume_sold",
    "Sold value": "revenue_sold"
})

# Drop unnamed or placeholder columns
df_sales_cleaned = df_sales_cleaned.drop(columns=[col for col in df_sales_cleaned.columns if "Unnamed" in col])


In [9]:
sales_out_path = os.path.join(output_folder, "sales_snapshot_cleaned.csv")
df_sales_cleaned.to_csv(sales_out_path, index=False)

print("Sales data cleaned and saved to:", sales_out_path)


Sales data cleaned and saved to: cleaned_data\sales_snapshot_cleaned.csv


## Clean Delivery File (Leveringen Filialen)

This file contains delivery volumes for each product per store on a specific day. It includes subgroup codes, article numbers, and the number of units delivered.

We apply the following column standardizations:

- `Filiaal` -> `store`
- `Datum` -> `date`
- `Artikel` -> `article`
- `Aantal Ontvangen CE` -> `delivered_quantity`


In [11]:
# Load delivery data
delivery_path = os.path.join(data_folder, "2025-01-24T06_03_32+00_00leveringen filialen c.e..xlsx")
df_delivery = pd.read_excel(delivery_path, sheet_name=0, skiprows=2)

# Rename columns
df_delivery_cleaned = df_delivery.rename(columns={
    "Filiaal": "store",
    "Datum": "date",
    "Artikel": "article",
    "Aantal Ontvangen CE": "delivered_quantity"
})

# Drop columns we don’t use (e.g., 'Subgroep', unnamed)
df_delivery_cleaned = df_delivery_cleaned.drop(columns=[col for col in df_delivery_cleaned.columns if "Unnamed" in col or "Subgroep" in col])


In [12]:
delivery_out_path = os.path.join(output_folder, "delivery_snapshot_cleaned.csv")
df_delivery_cleaned.to_csv(delivery_out_path, index=False)

print("Delivery data cleaned and saved to:", delivery_out_path)


Delivery data cleaned and saved to: cleaned_data\delivery_snapshot_cleaned.csv


## Clean Store Metadata (NAW Filialen)

This file contains branch-level metadata, including the name, address, and postal code of each store location.

We standardize the following columns:

- `Fil. Nr.` -> `store`
- `Filiaal` -> `store_name`
- `Adres` -> `address`
- `Postcode` -> `postal_code`
- `Plaatsnaam` -> `city`


In [13]:
# Load NAW filialen
naw_path = os.path.join(data_folder, "NAW filialen.xlsx")
df_naw = pd.read_excel(naw_path, sheet_name=0)

# Rename columns
df_naw_cleaned = df_naw.rename(columns={
    "Fil. Nr.": "store",
    "Filiaal": "store_name",
    "Adres": "address",
    "Postcode": "postal_code",
    "Plaatsnaam": "city"
})


In [14]:
naw_out_path = os.path.join(output_folder, "store_metadata_cleaned.csv")
df_naw_cleaned.to_csv(naw_out_path, index=False)

print("Store metadata cleaned and saved to:", naw_out_path)


Store metadata cleaned and saved to: cleaned_data\store_metadata_cleaned.csv


## Summary and Outputs

This notebook processed and standardized the following files:

- Waste data -> `waste_snapshot_cleaned.csv`
- Markdown (discount) data -> `markdown_snapshot_cleaned.csv`
- Sales data I -> `sales_snapshot_cleaned.csv`
- Delivery data -> `delivery_snapshot_cleaned.csv`
- Store metadata -> `store_metadata_cleaned.csv`

All files have been cleaned for:
- Header offsets
- Inconsistent or missing column names
- Irregular capitalization and formatting

Cleaned outputs are saved under the `cleaned_data/` folder and will serve as the foundation for merging, translation, semantic matching, and evaluation workflows in subsequent notebooks.
